In [52]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from IPython.core.display import Image, display
%matplotlib inline
import seaborn as sns
from matplotlib import pyplot as plt
import pylab as pl
import re
import codecs
import nltk
import pymorphy2
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pymorphy2
from nltk.probability import FreqDist

%pylab inline
pylab.rcParams['figure.figsize'] = (15,10)

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [72]:
df_p = pd.read_csv('../datasets/positive.csv',sep=';', header=None)
df_n = pd.read_csv('../datasets/negative.csv',sep=';', header=None)



In [31]:
df_n

,0,1,2,3,4,5,6,7,8,9,10,11
0,408906762813579264,1386325944,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,-1,0,0,0,8064,111,94,2
1,408906818262687744,1386325957,nugemycejela,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1,0,0,0,26,42,39,0
2,408906858515398656,1386325966,4post21,@elina_4post как говорят обещаного три года жд...,-1,0,0,0,718,49,249,0
3,408906914437685248,1386325980,Poliwake,"Желаю хорошего полёта и удачной посадки,я буду...",-1,0,0,0,10628,207,200,0
4,408906914723295232,1386325980,capyvixowe,"Обновил за каким-то лешим surf, теперь не рабо...",-1,0,0,0,35,17,34,0
...,...,...,...,...,...,...,...,...,...,...,...,...
111918,425138243257253888,1390195830,Yanch_96,Но не каждый хочет что то исправлять:( http://...,-1,0,0,0,1138,32,46,0
111919,425138339503943682,1390195853,tkit_on,скучаю так :-( только @taaannyaaa вправляет мо...,-1,0,0,0,4822,38,32,0
111920,425138437684215808,1390195876,ckooker1,"Вот и в школу, в говно это идти уже надо(",-1,0,0,1,165,13,16,0
111921,425138490452344832,1390195889,LisaBeroud,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :...",-1,0,1,0,2516,187,265,0


In [35]:
114911 + 111923

226834

In [73]:
df = pd.concat([df_p, df_n], ignore_index=True)
df

,0,1,2,3,4,5,6,7,8,9,10,11
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1
...,...,...,...,...,...,...,...,...,...,...,...,...
226829,425138243257253888,1390195830,Yanch_96,Но не каждый хочет что то исправлять:( http://...,-1,0,0,0,1138,32,46,0
226830,425138339503943682,1390195853,tkit_on,скучаю так :-( только @taaannyaaa вправляет мо...,-1,0,0,0,4822,38,32,0
226831,425138437684215808,1390195876,ckooker1,"Вот и в школу, в говно это идти уже надо(",-1,0,0,1,165,13,16,0
226832,425138490452344832,1390195889,LisaBeroud,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :...",-1,0,1,0,2516,187,265,0


In [37]:
df.iloc[114910:114912]

,0,1,2,3,4,5,6,7,8,9,10,11
114910,411368857035898880,1386912953,Prituljak_Sibir,@Ma_che_rie посмотри #непытайтесьпокинутьомск ...,1,0,0,0,1005,221,178,6
114911,408906762813579264,1386325944,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,-1,0,0,0,8064,111,94,2


In [74]:
df.set_axis(['id', 'date', 'name', 'text', 'positive', 'rep', 'rtv', 'fav', 'total_count', 'fol', 'friends', 'list_count'], axis=1, inplace=True)
df

,id,date,name,text,positive,rep,rtv,fav,total_count,fol,friends,list_count
0,408906692374446080,1386325927,pleease_shut_up,"@first_timee хоть я и школота, но поверь, у на...",1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,"Да, все-таки он немного похож на него. Но мой ...",1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,@irina_dyshkant Вот что значит страшилка :D\nН...,1,0,0,0,597,16,23,1
...,...,...,...,...,...,...,...,...,...,...,...,...
226829,425138243257253888,1390195830,Yanch_96,Но не каждый хочет что то исправлять:( http://...,-1,0,0,0,1138,32,46,0
226830,425138339503943682,1390195853,tkit_on,скучаю так :-( только @taaannyaaa вправляет мо...,-1,0,0,0,4822,38,32,0
226831,425138437684215808,1390195876,ckooker1,"Вот и в школу, в говно это идти уже надо(",-1,0,0,1,165,13,16,0
226832,425138490452344832,1390195889,LisaBeroud,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :...",-1,0,1,0,2516,187,265,0


In [75]:
print('Row count is:',len(df.index), 'pos:', len(df[df['positive'] == 1]), 'neg:', len(df[df['positive'] == -1]))

Row count is: 226834 pos: 114911 neg: 111923


In [76]:
df['text'] = df.text.str.lower()
print(df.text.loc[0:5])

df.text = df.text.str.replace(r"[^А-Яа-я]"," ")
df.text.loc[0:5]

0    @first_timee хоть я и школота, но поверь, у на...
1    да, все-таки он немного похож на него. но мой ...
2    rt @katiacheh: ну ты идиотка) я испугалась за ...
3    rt @digger2912: "кто то в углу сидит и погибае...
4    @irina_dyshkant вот что значит страшилка :d\nн...
5    ну любишь или нет? — я не знаю кто ты бля:d ht...
Name: text, dtype: object


0                 хоть я и школота  но поверь  у на...
1    да  все таки он немного похож на него  но мой ...
2                   ну ты идиотка  я испугалась за ...
3                     кто то в углу сидит и погибае...
4                    вот что значит страшилка    но...
5    ну любишь или нет    я не знаю кто ты бля     ...
Name: text, dtype: object

In [57]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> stopwords


      Unzipping corpora/stopwords.zip.



---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [77]:
df.text = list(map(word_tokenize, df.text))
df.text.loc[0:5]

0    [хоть, я, и, школота, но, поверь, у, нас, то, ...
1    [да, все, таки, он, немного, похож, на, него, ...
2           [ну, ты, идиотка, я, испугалась, за, тебя]
3    [кто, то, в, углу, сидит, и, погибает, от, гол...
4    [вот, что, значит, страшилка, но, блин, посмот...
5    [ну, любишь, или, нет, я, не, знаю, кто, ты, бля]
Name: text, dtype: object

In [78]:
def delete_stopword(words):
    russian_stopwords = stopwords.words("russian")
    new_s = [word for word in words if word not in russian_stopwords]
    return new_s

df.text = list(map(delete_stopword, df.text))
df.text.loc[0:5]

0    [школота, поверь, самое, общество, профилирующ...
1               [таки, немного, похож, мальчик, равно]
2                                [идиотка, испугалась]
3    [углу, сидит, погибает, голода, ещ, порции, вз...
4    [значит, страшилка, блин, посмотрев, части, со...
5                                  [любишь, знаю, бля]
Name: text, dtype: object

In [79]:
morph = pymorphy2.MorphAnalyzer()

def lemmatization(words):
    global morph
    new_s = [morph.parse(word)[0].normal_form for word in words]
    return new_s

df.text = list(map(lemmatization, df.text))
df.text.loc[0:5]


0    [школотый, поверь, самый, общество, профилиров...
1             [таки, немного, похожий, мальчик, равно]
2                                [идиотка, испугаться]
3    [угол, сидеть, погибать, голод, ещ, порция, вз...
4    [значит, страшилка, блин, посмотреть, часть, с...
5                                 [любить, знать, бля]
Name: text, dtype: object

1493275

In [ ]:
"""1493275   – общее количество слов перед удалением слов, встречающихся 1 раз
53568   – количество слов, встречающихся 1 раз
1439707   – итоговое количество слов
856   – количество пустых твитов
382   – из них позитивных твитов
225978   – количество твитов после удаления пустых"""

In [81]:
def to_str(s):
    new_s = ' '.join(j for j in s)
    return new_s

text_tokens = word_tokenize(' '.join(j for j in list(map(to_str, df.text))))
text = nltk.Text(text_tokens)
fdist = FreqDist(text)
words_to_del = set(list(filter(lambda k: fdist[k] == 1, fdist)))

def delete_word(words):
    global words_to_del
    new_s = [word for word in words if word not in words_to_del]
    return new_s
df.text = list(map(delete_word, df.text))
df.text = list(map(to_str, df.text))
df.text.loc[0:5]

53568
1439707


0           школотый поверь самый общество предмет тип
1                   таки немного похожий мальчик равно
2                                   идиотка испугаться
3    угол сидеть погибать голод ещ порция взять хот...
4    значит страшилка блин посмотреть часть создать...
5                                     любить знать бля
Name: text, dtype: object

In [86]:
print("all_w: ", len(text_tokens),"to_del: ", len(words_to_del), "itog: ", len(text_tokens) - len(words_to_del))

all_w:  1493275 to_del:  53568 itog:  1439707


In [87]:
len(df[df.text == '']), len(df[(df.text == '') & (df.positive == 1)])

(856, 382)

In [88]:
df = df.drop(df[df.text == ''].index, axis = 0)
len(df)

225978

In [89]:
df

,id,date,name,text,positive,rep,rtv,fav,total_count,fol,friends,list_count
0,408906692374446080,1386325927,pleease_shut_up,школотый поверь самый общество предмет тип,1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,таки немного похожий мальчик равно,1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,идиотка испугаться,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,угол сидеть погибать голод ещ порция взять хот...,1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,значит страшилка блин посмотреть часть создать...,1,0,0,0,597,16,23,1
...,...,...,...,...,...,...,...,...,...,...,...,...
226829,425138243257253888,1390195830,Yanch_96,каждый хотеть исправлять,-1,0,0,0,1138,32,46,0
226830,425138339503943682,1390195853,tkit_on,скучать вправлять мозг равно скучать,-1,0,0,0,4822,38,32,0
226831,425138437684215808,1390195876,ckooker1,школа говно это идти,-1,0,0,1,165,13,16,0
226832,425138490452344832,1390195889,LisaBeroud,тауриэль грусть обнять,-1,0,1,0,2516,187,265,0


In [98]:
df.to_csv('after_all.csv')
df

,id,date,name,text,positive,rep,rtv,fav,total_count,fol,friends,list_count
0,408906692374446080,1386325927,pleease_shut_up,школотый поверь самый общество предмет тип,1,0,0,0,7569,62,61,0
1,408906692693221377,1386325927,alinakirpicheva,таки немного похожий мальчик равно,1,0,0,0,11825,59,31,2
2,408906695083954177,1386325927,EvgeshaRe,идиотка испугаться,1,0,1,0,1273,26,27,0
3,408906695356973056,1386325927,ikonnikova_21,угол сидеть погибать голод ещ порция взять хот...,1,0,1,0,1549,19,17,0
4,408906761416867842,1386325943,JumpyAlex,значит страшилка блин посмотреть часть создать...,1,0,0,0,597,16,23,1
...,...,...,...,...,...,...,...,...,...,...,...,...
226829,425138243257253888,1390195830,Yanch_96,каждый хотеть исправлять,-1,0,0,0,1138,32,46,0
226830,425138339503943682,1390195853,tkit_on,скучать вправлять мозг равно скучать,-1,0,0,0,4822,38,32,0
226831,425138437684215808,1390195876,ckooker1,школа говно это идти,-1,0,0,1,165,13,16,0
226832,425138490452344832,1390195889,LisaBeroud,тауриэль грусть обнять,-1,0,1,0,2516,187,265,0


In [90]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.text, df.positive, test_size=0.3, random_state=21)

In [91]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

cv = CountVectorizer()
cv_train = cv.fit_transform(X_train)
cv_test = cv.transform(X_test)

tfidf = TfidfVectorizer()
tfidf_train = tfidf.fit_transform(X_train)
tfidf_test = tfidf.transform(X_test)

In [92]:
lr_ms = LogisticRegression(random_state=21)
lr_ms.fit(cv_train, y_train)
cv_pred = lr_ms.predict(cv_test)
print('test')
print(classification_report(y_test, cv_pred))
print('train')
print(classification_report(y_train, lr_ms.predict(cv_train)))

test
              precision    recall  f1-score   support

          -1       0.72      0.71      0.71     33422
           1       0.72      0.73      0.72     34372

    accuracy                           0.72     67794
   macro avg       0.72      0.72      0.72     67794
weighted avg       0.72      0.72      0.72     67794

train
              precision    recall  f1-score   support

          -1       0.80      0.79      0.79     78027
           1       0.80      0.80      0.80     80157

    accuracy                           0.80    158184
   macro avg       0.80      0.80      0.80    158184
weighted avg       0.80      0.80      0.80    158184



In [93]:
lr_tf = LogisticRegression(random_state=21)
lr_tf.fit(tfidf_train, y_train)
tfidf_pred = lr_tf.predict(tfidf_test)
print('test')
print(classification_report(y_test, tfidf_pred))
print('train')
print(classification_report(y_train, lr_tf.predict(tfidf_train)))

test
              precision    recall  f1-score   support

          -1       0.73      0.69      0.71     33422
           1       0.71      0.75      0.73     34372

    accuracy                           0.72     67794
   macro avg       0.72      0.72      0.72     67794
weighted avg       0.72      0.72      0.72     67794

train
              precision    recall  f1-score   support

          -1       0.79      0.75      0.77     78027
           1       0.77      0.80      0.79     80157

    accuracy                           0.78    158184
   macro avg       0.78      0.78      0.78    158184
weighted avg       0.78      0.78      0.78    158184



In [94]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=21)
forest.fit(cv_train, y_train)
cv_pred = forest.predict(cv_test)
print('test')
print(classification_report(y_test, cv_pred))
print('train')
print(classification_report(y_train, forest.predict(cv_train)))

test
              precision    recall  f1-score   support

          -1       0.68      0.76      0.72     33422
           1       0.74      0.65      0.69     34372

    accuracy                           0.70     67794
   macro avg       0.71      0.71      0.70     67794
weighted avg       0.71      0.70      0.70     67794

train
              precision    recall  f1-score   support

          -1       1.00      1.00      1.00     78027
           1       1.00      1.00      1.00     80157

    accuracy                           1.00    158184
   macro avg       1.00      1.00      1.00    158184
weighted avg       1.00      1.00      1.00    158184



In [95]:
forest.fit(tfidf_train, y_train)
tfidf_pred = forest.predict(tfidf_test)
print('test')
print(classification_report(y_test, tfidf_pred))
print('train')
print(classification_report(y_train, forest.predict(tfidf_train)))

test
              precision    recall  f1-score   support

          -1       0.72      0.70      0.71     33422
           1       0.72      0.74      0.73     34372

    accuracy                           0.72     67794
   macro avg       0.72      0.72      0.72     67794
weighted avg       0.72      0.72      0.72     67794

train
              precision    recall  f1-score   support

          -1       1.00      1.00      1.00     78027
           1       1.00      1.00      1.00     80157

    accuracy                           1.00    158184
   macro avg       1.00      1.00      1.00    158184
weighted avg       1.00      1.00      1.00    158184

